**Note:** This notebook is written in the Julia language, so the cells can't be
  executed in Google Colab. If you want to verify that the notebook works, I
  recommend [JuliaBox](https://juliabox.com/) or testing locally. The syntax is
  very similar to Python and MATLAB. Note in particular the dot syntax used to
  perform elementwise operations (`f.(x)` applies `f` to all elements of `x`)
  and that indices start at 1.

# **Lab 2: Direct methods**
**Anders Ågren Thuné**

# **Abstract**

Short summary of the lab report. State the objectives, methods used, main
results and conlusions.

# *About the code**

A short statement on who is the author of the file, and if the code is
distributed under a certain license.

In [1]:
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019
# Anders Ågren Thuné (athune@kth.se)
# Johan Hoffman (jhoffman@kth.se)

# Code written by Anders Ågren Thuné based on the template by Johan Hoffman.

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [1]:
using LinearAlgebra
import Base: getproperty

# **Introduction**

Systems of linear equations appear frequently in a wide variety of problems. A
system of linear equations can be expressed in linear algebra terms as $Ax = b$,
which, if solvable, has the solution $x = A^{-1}b$. As such, being able to
compute the inverse of a given matrix is a problem of great importance. This is
difficult in general, but certain classes of matrices have easily computable
inverses; an orthogonal matrix $Q$ has the inverse $Q^T$ and the inverse of a
triangular matrix can be computed through back (or forward) substitution.
Therefore, a common approach to computing the inverse of a given matrix is to
factor it into triangular and orthogonal matrices, which can then be easily
inverted. One such factorization is the QR factorization in which any matrix $A$
is factored into an orthogonal matrix $Q$ and an upper triangular matrix $R$.
The inverse is then given by $A^{-1}=(QR)^{-1}=R^{-1}Q^T$.

Even when the system $Ax=b$ is unsolvable (there is no $A^{-1}), the best
possible solution can be obtained by projecting $b$ orthogonally onto
$range(A)$. This gives the approximated solution $\hat{x} = (A^TA)^{-1}A^Tb$,
where $(A^TA)^{-1}$ is called the *pseudo-inverse* of $A$.

The QR factorization can also be used in a wider range of problems. One such
problem is eigenvalue computation, where an iterative $QR$-algorithm can be used
to obtain a Schur factorization of a symmetric matrix. That is, a factorization
$A = QTQ^T$, where $T$ is a triangular matrix with the eigenvalues of $A$ on
the diagonal.

This report presents how these concepts, described in chapters 5 and 6 of the
lecture notes, were used to implement the following:
- A function for QR factorization
- A direct solver of of $Ax=b$
- A solver of the least squares problem $Ax=b$
- A function performing the QR eigenvalue algorithm

# **Methods**

## **QR factorization**

The QR factorization for a matrix can be calculated in a number of different
ways. The method implemented here is the *Householder QR factorization*, based
on Householder reflections. A matrix $P$ of the form $P = I-\beta vv^T, \quad
\beta = \frac{2}{v^Tv}$ is a *Householder reflection*. This is an orthogonal
reflector reflecting a given vector $x$ in the hyperplane
$\text{span}\{v\}^\perp. Selecting $v$ = $\pm \|x\|$ gives $Px = \pm\|x\|e_1$,
which can be utilized to construct $Q_n\dots Q_2Q_1A = R$, where each matrix
$Q_k$ is constructed to zero the subdiagonal elements of a column of $A$. This
is achieved by letting $Q_k = \begin{matrix} I & 0 \\ 0 P \end{matrix}$, where P
is a Householder reflection. When dealing with a Householder QR factorization,
it is advantageous not to explicitly form the matrices $Q$ or $Q_k$, but instead
utilize the fact that $Q_kA = A - (\beta v)(v^TA)$, and that $Q=Q_1^TQ_2^T\dots
Q_n^T$.

[Golub & Van Loan 2013: Chapter 5.1-5.2, Hoffman 2019: Chapter 5.2]

The following implementation is based on Algorithm 5.2.1 of Golub & Van Loan
(2013), and results in a matrix with the upper triangular part of $R$ as its
upper triangular part, and the vectors $v_j$ required to construct $Q$ below the
diagonal. To be precise, all elements except the first of these vectors are
stored, with the implicit assumption that they are normalized such that the
first element is always 1. In addition to the function itself, a wrapper
struct (analogous to a Python class) is implemented to facilitate easy handling
of this format.

In [180]:
mutable struct QRfact{T <: AbstractFloat}
    QR :: Matrix{T}
    betas :: Vector{T}

    function QRfact(A :: Matrix{T}) where T <: AbstractFloat
        (m, n) = size(A)
        v = zeros(T, m)
        betas = zeros(T, n)

        for j = 1 : n
            vj = v[j:m]
            vj .= A[j:m,j]
            vj[1] += sign(vj[1])*norm(vj)
            betas[j] = 2vj[1]^2/norm(vj)^2
            vj ./= -vj[1]
            for k = j : n
                A[j:m,k] .-= betas[j].*vj.*dot(vj,A[j:m,k])
            end
            if j<m
               A[j+1:m,j] .= vj[2:end]
            end
        end
        new{T}(A, betas)
    end
end

function getproperty(qr :: QRfact{T}, f :: Symbol) where T
    QR = getfield(qr, :QR)
    m = size(QR)[1]
    if f == :q
        Q = Matrix{T}(I,m,m)
        mulQ!(qr,Q, fromright = true)
        return Q
    elseif f == :qt
        Qt = Matrix{T}(I,m,m)
        mulQ!(qr,Qt,transpose=true)
        return Qt
    elseif f == :r
        return UpperTriangular(copy(QR))
    elseif f == :r_mutate
        return UpperTriangular(QR)
    else
        error("type $(typeof(qr)) has no field $f")
    end
end

function mulQ!(qr :: QRfact{T}, C :: Matrix{T};
               transpose = false, fromright = false) where T
    QR = getfield(qr, :QR)
    betas = getfield(qr, :betas)
    (m, n) = size(QR)
    iterRange = xor(transpose, fromright) ? (n-1:-1:1) : (1:n-1)
    v = zeros(T, m)
    temp = zeros(T,m)
    for j = iterRange
        v[1:j-1] .= zero(T)
        v[j] = one(T)
        v[j+1:m] .= QR[j+1:m,j]
        if fromright
            mul!(temp, C, v)
            for k = 1 : n
                C[:,k] .-= temp .*betas[j].*v[k]
            end
        else
            for k = 1 : n
                C[:,k] .-= betas[j].*v.*dot(v,C[:,k])
            end
        end
    end
end

QRfactorization(A :: Matrix{<:AbstractFloat}) = QRfact(copy(A))

QRfactorization (generic function with 1 method)

In [175]:
u = UpperTriangular(A)

3×3 UpperTriangular{Float64,Array{Float64,2}}:
 1.0  3.0  4.0
  ⋅   6.0  2.0
  ⋅    ⋅   3.0

In [176]:
xor(false, false)

false

In [177]:
A = [1.0 3 4; 4 6 2; 1 1 3]
test = QRfactorization(A);

In [178]:
orth = test.qt

3×3 Array{Float64,2}:
 -0.235702  -0.673292   0.700802 
  0.942809  -0.333319  -0.0031369
  0.235702   0.659983   0.713349 

In [179]:
orth * orth'

3×3 Array{Float64,2}:
  1.0           1.90386e-16  -5.55112e-17
  1.90386e-16   1.0          -8.37004e-17
 -5.55112e-17  -8.37004e-17   1.0        

# **Results**

Present the results. If the result is an algorithm that you have described under
the *Methods* section, you can present the data from verification and
performance tests in this section. If the result is the output from a
computational experiment this is where you present a selection of that data.

# **Discussion**

Summarize your results and your conclusions. Were the results expected or
surprising. Do your results have implications outside the particular problem
investigated in this report?

# **References**

- Hoffman, J. 2019. *Introduction to Scientific Computing*
- Golub, Gene H. and Van Loan, Charles F. 2013. *Matrix Computations*. 4th ed. Baltimore: John Hopkins University Press.